In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib

from datetime import datetime
from google.cloud import secretmanager

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

In [ ]:
con = pesys_utils.open_pen_connection()
df_v7 = pandas_utils.pandas_from_sql('../sql/v7.sql', con)
con.close()

In [ ]:
df = df_v7.copy()

In [ ]:
# Kan vurdere å sortere nullverdier i auto/man:
df["KRAV_BEHANDLING"] = df.KRAV_BEHANDLING.fillna("USPESIFISERT")
df["MAANED"] = df.MAANED.astype('string').apply(lambda x: pesys_utils.add_zero_to_mnd(x))
df["AAR_MAANED"] = df.AAR_MAANED.apply(lambda x: pesys_utils.add_zero_to_aar_mnd(x))

In [ ]:
current_aar_maaned = datetime.today().strftime('%Y-%m')
df = df[df.AAR_MAANED != current_aar_maaned]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'pensjon-saksbehandli-prod-1f83.vedtak.vedtak_automatisering'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()

## Alle stønadsområder
- AFP
- AFP Privat
- Alderspensjon
- Barnepensjon
- Familiepleierytelse
- Gammel yrkesskade
- Generell
- Gjenlevendeytelse
- Krigspensjon
- Omsorgsopptjening
- Uføretrygd

### Mulige kravtyper (De fleste i hvert fall)
- AFP etteroppgjør
- Anke
- Eksport
- Endring uttaksgrad
- Erstatning
- Ettergivelse av gjeld
- Førstegangsbehandling
- Førstegangsbehandling Norge/utland
- Førstegangsbehandling bosatt utland'
- Førstegangsbehandling kun utland
- Godskriving omsorgsopptjening
- Hjelpeberegning ved overgang til uføretrygd
- Inntektsendring
- Klage
- Mellombehandling
- Omgjøring av tilbakekreving
- Overføring omsorgsopptjening
- ReguleringRevurdering
- Saksomkostninger
- Sluttbehandling Norge/utland
- Sluttbehandling kun utland
- Søknad om reduksjon av uføregrad
- Søknad om ung ufør
- Søknad om yrkesskade
- Søknad om økning av uføregrad
- Tilbakekreving
- Uføretrygd etteroppgjør
- Uføretrygd vurdering av etteroppgjør
- Utsendelse til avtaleland